In [0]:
# Initial Commit 2

# i (with a 73%) confidence tell that the customer has a 86% chance of not renewing the subscription 

In [0]:
df = spark.read.format("csv").option("header", "true").load("/Volumes/databricks_telco_customer_dataset/v01/telco/telco-customer-churn.csv")
display(df)

In [0]:

import os
import joblib
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    classification_report,
    confusion_matrix,
)

# ---------- 1. Load data ----------
DATA_PATH = "/Volumes/databricks_telco_customer_dataset/v01/telco/telco-customer-churn.csv"
df = pd.read_csv(DATA_PATH) ##historic data
# df

In [0]:
# ---------- 2. Basic cleaning ----------

# Convert TotalCharges to numeric (it comes as string with some blanks)
if "TotalCharges" in df.columns:
    df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")
    # Drop rows where TotalCharges can't be converted (usually a tiny number)
    df = df.dropna(subset=["TotalCharges"])

# Drop customerID – it's just an identifier
if "customerID" in df.columns:
    df = df.drop(columns=["customerID"])

# Target column
TARGET_COL = "Churn"
y = df[TARGET_COL].map({"Yes": 1, "No": 0})  # binary 0/1
X = df.drop(columns=[TARGET_COL])
display(X)